In [1]:
!nvidia-smi


Thu Jan 22 20:32:54 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
ZIP_PATH = "/content/drive/MyDrive/datasets/BKDN-HUYLV-DATN.v1i.yolov8.zip"

In [4]:
import os
print("Zip found:", os.path.exists(ZIP_PATH))


Zip found: True


In [5]:
!pip install -q ultralytics
!yolo checks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 40.7 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.7/112.6 GB disk)

OS                     Linux-6.6.105+-x86_64-with-glibc2.35
Environment            Colab
Python                 3.12.12
Install                pip
Path                   /usr/local/lib/python3.12/dist-packages/ultralytics
RAM                    12.67 GB
Disk                   38.7/112.6 GB
CPU                    Intel Xeon CPU @ 2.00GHz
CPU count              2
GPU                    Tesla T4, 15095MiB
GPU count              1
CUDA                   12.6

numpy      

In [6]:
import os, zipfile

dataset_dir = "/content/dataset"
os.makedirs(dataset_dir, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, "r") as z:
    z.extractall(dataset_dir)

print("✅ Unzipped to:", dataset_dir)
print("Top files:", os.listdir(dataset_dir)[:15])

✅ Unzipped to: /content/dataset
Top files: ['README.roboflow.txt', 'test', 'data.yaml', 'train', 'README.dataset.txt', 'valid']


In [7]:
import yaml, os

yaml_path = os.path.join(dataset_dir, "data.yaml")

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

data["path"] = dataset_dir
data["train"] = "train/images"
data["val"]   = "valid/images"
data["test"]  = "test/images"

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f, sort_keys=False)

print("✅ data.yaml fixed!")


✅ data.yaml fixed!


In [8]:
!yolo task=detect mode=train model=yolov8s.pt data=/content/dataset/data.yaml imgsz=640 epochs=100 batch=16

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, 

In [9]:
# Evaluate on validation split
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/dataset/data.yaml

# Evaluate on test split
!yolo task=detect mode=val model=/content/runs/detect/train/weights/best.pt data=/content/dataset/data.yaml split=test

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2677.9±509.9 MB/s, size: 100.5 KB)
val: Scanning /content/dataset/valid/labels.cache... 335 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 335/335 70.3Mit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 21/21 3.1it/s 6.7s
                   all        335        435      0.908      0.838      0.888      0.631
                  Hole         92        122      0.975      0.955      0.973      0.824
                  Knot         87        103      0.926      0.806      0.874      0.559
                  Line         90        101      0.837      0.743      0.795      0.487
                 Stain         78        109      0.894      0.847      0.909      0.652
Speed: 3.9ms preprocess, 8.8ms inference, 0.0ms 

In [10]:
!yolo task=detect mode=predict model=/content/runs/detect/train/weights/best.pt source=/content/dataset/test/images conf=0.25 save=True



Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 73 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs

image 1/188 /content/dataset/test/images/10_jpg.rf.ae9b71499dc44cbb8f3a59ed095063ba.jpg: 640x640 1 Knot, 16.3ms
image 2/188 /content/dataset/test/images/13_jpg.rf.7c31f11ea3c8743d19653aa21d0a5065.jpg: 640x640 1 Knot, 16.2ms
image 3/188 /content/dataset/test/images/20_jpg.rf.52c5bb7fc47b532cd723e9e0bd1f5ea8.jpg: 640x640 1 Stain, 16.2ms
image 4/188 /content/dataset/test/images/23_jpg.rf.7b7ef51112a2068d648e1ba1984070fb.jpg: 640x640 1 Hole, 16.2ms
image 5/188 /content/dataset/test/images/4_jpg.rf.29bcc3578ff425c50875bf548944a4b4.jpg: 640x640 1 Line, 16.2ms
image 6/188 /content/dataset/test/images/8_jpg.rf.b3c5d0472da534b6121b3aee9e6e3dba.jpg: 640x640 1 Knot, 1 Line, 16.2ms
image 7/188 /content/dataset/test/images/train_00007_jpg.rf.0bdd42e87535035824857bbfd29eee03.jpg: 640x640 1 Line, 16.2ms
image 8/188 /content/dataset/test/im

In [11]:
from google.colab import drive
import os, shutil

drive.mount('/content/drive')

SAVE_DIR = "/content/drive/MyDrive/yolov8_results"
os.makedirs(SAVE_DIR, exist_ok=True)

# 1) Copy best/last weights
best_src = "/content/runs/detect/train/weights/best.pt"
last_src = "/content/runs/detect/train/weights/last.pt"

if os.path.exists(best_src):
    shutil.copy(best_src, os.path.join(SAVE_DIR, "best.pt"))
if os.path.exists(last_src):
    shutil.copy(last_src, os.path.join(SAVE_DIR, "last.pt"))

# 2) Copy whole training folder (includes results.csv, results.png, curves, confusion matrix)
train_dir = "/content/runs/detect/train"
if os.path.exists(train_dir):
    shutil.make_archive(os.path.join(SAVE_DIR, "detect_train"), 'zip', train_dir)

# 3) Copy latest predict folder (predict, predict2, ...)
predict_base = "/content/runs/detect"
predict_folders = sorted([d for d in os.listdir(predict_base) if d.startswith("predict")])

if predict_folders:
    latest_predict = os.path.join(predict_base, predict_folders[-1])
    shutil.make_archive(os.path.join(SAVE_DIR, f"{predict_folders[-1]}"), 'zip', latest_predict)

# 4) Backup whole runs folder (everything)
runs_dir = "/content/runs"
if os.path.exists(runs_dir):
    shutil.make_archive(os.path.join(SAVE_DIR, "runs_backup"), 'zip', runs_dir)

print("✅ Saved to Drive folder:", SAVE_DIR)
print("Files saved:", os.listdir(SAVE_DIR))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Saved to Drive folder: /content/drive/MyDrive/yolov8_results
Files saved: ['best.pt', 'last.pt', 'detect_train.zip', 'predict.zip', 'runs_backup.zip']


In [12]:
import os
print(os.listdir("/content/drive/MyDrive/yolov8_results"))


['best.pt', 'last.pt', 'detect_train.zip', 'predict.zip', 'runs_backup.zip']
